In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import pickle as pkl

from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv('data/ETHBTC_1519419600000_1549491000000.csv')
df.head(10)

OPEN_TIME     OPEN     HIGH      LOW    CLOSE       VOLUME  \
0  1519419540000  8359800  8361800  8356800  8361600   8419500000   
1  1519419600000  8361600  8373100  8357400  8365800  13991600000   
2  1519419660000  8364900  8375600  8360000  8370000  11501500000   
3  1519419720000  8370100  8384200  8367300  8373700  10797600000   
4  1519419780000  8371700  8376600  8365000  8373600   8432200000   
5  1519419840000  8374700  8385500  8371500  8374300   7674500000   
6  1519419900000  8375000  8377700  8372500  8374000   4918300000   
7  1519419960000  8374000  8385900  8374000  8377100   5988700000   
8  1519420020000  8376400  8386400  8375200  8383700   8866300000   
9  1519420080000  8383700  8385800  8379000  8385300   7171100000   

      CLOSE_TIME   QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1519419599999   703922544         222     5724200000       478637316      0  
1  1519419659999  1170291760         559     8443400000       706266888      0  
2  1519419719999   962386071         381     4916200000       411444581      0  
3  1519419779999   904137433         551     4966800000       415973892      0  
4  1519419839999   705985415         437     4726600000       395780160      0  
5  1519419899999   642856897         442     4029400000       337578135      0  
6  1519419959999   411898485         403     2276200000       190648794      0  
7  1519420019999   501696773         667     2554100000       214022640      0  
8  1519420079999   742844782         483     2584400000       216589789      0  
9  1519420139999   601094149         272     3257300000       273066651      0

In [4]:
data = df.loc[:, ['OPEN_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']].values
data.shape
data[:5]

(500000, 5)

array([[1519419540000,       8359800,       8361800,       8356800,
              8361600],
       [1519419600000,       8361600,       8373100,       8357400,
              8365800],
       [1519419660000,       8364900,       8375600,       8360000,
              8370000],
       [1519419720000,       8370100,       8384200,       8367300,
              8373700],
       [1519419780000,       8371700,       8376600,       8365000,
              8373600]])

In [5]:
### Data Transformation

## Params
window = 100
#########

num_samples = data.shape[0] - window + 1

data_trans = np.zeros((num_samples, window*(data.shape[1]-1)))
for i in range(num_samples):
    data_trans[i] = data[i:i + window, 1:].flatten()

data_trans = np.hstack((data[window-1:window+num_samples-1, 0][np.newaxis].T, data_trans))

data_trans.shape

(499901, 401)

In [6]:
pca = PCA(n_components=10, whiten=True, random_state=0)
pca = pca.fit(data_trans[:, 1:])

In [7]:
pca.explained_variance_ratio_.shape
pca.explained_variance_ratio_.sum()
pca.explained_variance_ratio_[:10]

(10,)

0.9999951238374001

array([9.99932681e-01, 4.05682751e-05, 1.05112449e-05, 4.52611959e-06,
       2.44446112e-06, 1.50567767e-06, 1.04225790e-06, 7.66361487e-07,
       5.94592937e-07, 4.83940012e-07])

In [8]:
data_to_dump = {
    'pca_model': pca
}

In [9]:
data_pca = pca.transform(data_trans[:, 1:])

data_pca = np.hstack((data_trans[:, 0][np.newaxis].T, data_pca))

df_cols = ['OPEN_TIME']
df_cols.extend(['PCA_'+str(i) for i in range(data_pca.shape[1]-1)])

data_pca = pd.DataFrame(data_pca, columns=df_cols)

data_pca.shape

(499901, 11)

In [10]:
data_to_dump['data_pca'] = data_pca

In [11]:
data_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'intv':'1m', 
    'cols':'OHCL', 
    'win':100}.iteritems()])

proc_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'ncomp':10, 
    'expvar':'{:.4f}'.format(pca.explained_variance_ratio_.sum())}.iteritems()])

import time
time_tag = time.strftime('%y%m%d_%H%M%S')

big_tag = time_tag+'_|_'+data_tag+'_|_'+proc_tag

f = open('pca_models/'+time_tag+'.pkl', 'wb')
pkl.dump(data_to_dump, f, -1)
f.close()

%notebook -e pca_models/{big_tag}.ipynb